Collect and Store Sharpe Values

In [1]:
import time
import os 

start_time = time.time()

WORKSPACE_DIR = os.getenv('WORKSPACE_DIR')

if not os.getcwd().endswith('portfolio_py'):
    os.chdir(f'{WORKSPACE_DIR}/portfolio_py')
print(f'Current Working Directory: {os.getcwd()}')

from utils.finance_utils import calculate_sharpe_ratio
from utils.helpers import divide_chunks
from utils.config import PROGRAM_START_DATE, PROGRAM_END_DATE

Current Working Directory: /Users/blakeuribe/Desktop/portfolio_py

---------------------------------
finance_utils.py successfully loaded, updated last Feb. 07 2025 3.11
---------------------------------



---------------------------------
helpers.py successfully loaded, updated last Feb. 04 2025
---------------------------------


Updated on 02/10/2025 8:54


In [2]:
from dotenv import load_dotenv

import yfinance as yf
import numpy as np
import pandas as pd

from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
load_dotenv()

print(f'Ending Program at: {PROGRAM_END_DATE}')
print(f'Starting Program at: {PROGRAM_START_DATE}')

Ending Program at: 2025-01-01
Starting Program at: 2024-01-01


Begin Collecting Supportive Data in Calculations

In [4]:
# Download and import data
ticker_df = pd.read_json(f'{WORKSPACE_DIR}/portfolio_py/data/raw/company_tickers.json')
ticker_df = ticker_df.T.drop(columns=['cik_str', 'title'])

tbill_data = yf.download('^IRX', start=PROGRAM_START_DATE, end=PROGRAM_END_DATE, auto_adjust=True)['Close']
tbill_data = tbill_data / 100 / 360  # Convert to daily rate

spy_sharpe = calculate_sharpe_ratio(np.array('spy'), tbill=tbill_data, start_date=PROGRAM_START_DATE, end_date=PROGRAM_END_DATE)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Begin Calculating Sharpe Values

In [5]:

# Set chunk size and number of stocks
num_in_chunks = 15
num_of_stocks_to_get = 500  # Adjust as needed

# Get tickers and divide into chunks

print('----Dividing Into Chunks----')
tickers = ticker_df['ticker'][0:num_of_stocks_to_get]  # Remove .tolist() here
ticker_chunks = list(divide_chunks(tickers, num_in_chunks))

print(f'Divided {num_of_stocks_to_get} tickers into {len(ticker_chunks)} chunks')

# Initialize an empty list to store results
df_list = []

print('\n----Collecting Sharpe Ratios----')
# Loop through each chunk and process the stocks
for chunk in ticker_chunks:
    try:
        # Calculate Sharpe ratios for the current chunk
        sharpe_ratios_series = calculate_sharpe_ratio(
            chunk, 
            tbill=tbill_data, 
            start_date=PROGRAM_START_DATE, 
            end_date=PROGRAM_END_DATE
        )
        # Create a DataFrame for the chunk
        results_df_chunk = pd.DataFrame({
            'Tickers': sharpe_ratios_series.index, 
            'Sharpe_ratios': sharpe_ratios_series.values
        })
        df_list.append(results_df_chunk)

    except Exception as e:
        print(f'Error calculating Sharpe ratios for chunk: {chunk}\n{str(e)}')

# Concatenate all DataFrames vertically
final_df = pd.concat(df_list, axis=0, ignore_index=True)


# Ensure we have a benchmark value for futre use
final_df.loc[len(final_df)] = ['SPY', float(spy_sharpe.values[0])]

final_df = final_df.drop_duplicates()

# Check percentage of NaN values
pct_of_nan = (final_df.isna().sum().sum() / len(final_df) * 100).round(2)
print('\n----Df Report----')
print(f'Pct of NaN values is: {pct_of_nan}%')

# Save the results to CSV

if pct_of_nan >= 25:
    # Ask the user if they still want to export
    user_input = input(f"The percentage of NaN values is {pct_of_nan:.2f}%. Do you still want to export the DataFrame? (yes/no): ").strip().lower()

    if user_input == 'yes':
        final_df.to_csv(f'{WORKSPACE_DIR}/portfolio_py/data/clean/sharpe_ratios.csv', index=False)
        print('Df containing Sharpe Ratios Successfully Exported')
    else:
        print('Export cancelled.')
else:
    final_df.to_csv(f'{WORKSPACE_DIR}/portfolio_py/data/clean/sharpe_ratios.csv', index=False)
    print('Nan Values are below threshold, Successfully Exported')


end_time = time.time()
elapsed_time = end_time - start_time

print('\n----Time Report----')
print(f'Processing time: {elapsed_time:.2f} seconds, for {num_of_stocks_to_get} Tickers')

[*******************   40%                       ]  6 of 15 completed

----Dividing Into Chunks----
Divided 500 tickers into 34 chunks

----Collecting Sharpe Ratios----


[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  15 of 15 completed
[*****


----Df Report----
Pct of NaN values is: 3.0%
Nan Values are below threshold, Successfully Exported

----Time Report----
Processing time: 15.69 seconds, for 500 Tickers


In [6]:
final_df

,Tickers,Sharpe_ratios
0,AAPL,1.577152
1,AMZN,1.646417
2,AVGO,1.954486
3,BRK-B,1.651065
4,GOOGL,1.380955
...,...,...
495,AWK,-0.374826
496,DTE,0.626060
497,VLTO,1.311954
498,VTR,1.003038
